# Finding out why Monte Carlo results are significantly different than static ones

Based on a [Stack Overflow question](https://stackoverflow.com/questions/72807629/overestimated-monte-carlo-results-in-brightway).

Uses Brightway 2.5.

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
import numpy as np
import pandas as pd

In [2]:
bd.projects.set_current("ecoinvent 3.8 consequential 25")

Define functional unit and LCIA impact category

In [3]:
waste_paper = bd.Database("ecoinvent 3.8 consequential").get(name='market for waste paper, sorted')
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

Create static and stochastic LCA instances

In [4]:
static = bc.LCA({waste_paper: 1}, ipcc)
static.lci()
static.lcia()

In [5]:
mc = bc.LCA({waste_paper: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

Confirm that the scores are quite different

In [6]:
static.score, np.mean([mc.score for _ in zip(range(20), mc)])

(2.76826441779881, 140.97417359119535)

We can do a recursive calculation on the Monte Carlo object (this doesn't call `__next__`, so the matrices remain the same), but we need to give some normally private arguments:

In [7]:
ba.print_recursive_calculation(
    waste_paper,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.01,
    _lca_obj=mc,
    _total_score=mc.score,
)

Fraction of score | Absolute score | Amount | Activity
0001 | 36.23 |     1 | 'market for waste paper, sorted' (kilogram, GLO, None)
  0.288 | 10.42 | 0.2857 | 'tissue paper production' (kilogram, RER, None)
    0.297 | 10.77 | 0.2434 | 'market for tissue paper' (kilogram, GLO, None)
      0.297 | 10.75 | 0.2434 | 'tissue paper production, virgin' (kilogram, GLO, None)
        0.0119 | 0.4314 | 0.247 | 'market for sulfite pulp, bleached' (kilogram, GLO, None)
  0.712 | 25.81 | 0.7143 | 'tissue paper production' (kilogram, RoW, None)
    0.743 | 26.93 | 0.6084 | 'market for tissue paper' (kilogram, GLO, None)
      0.742 | 26.87 | 0.6084 | 'tissue paper production, virgin' (kilogram, GLO, None)
        0.0165 | 0.5968 | 4.327 | 'market group for heat, district or industrial, other than natural gas
          0.0131 | 0.4762 | 3.586 | 'market for heat, district or industrial, other than natural gas' (meg
        0.0195 | 0.7081 | 1.217 | 'market group for electricity, medium voltage' (kil

We can see from this that there is a large impact from `tissue paper production, virgin`. Let's switch our functional unit to this:

In [8]:
tissue = bd.Database("ecoinvent 3.8 consequential").get(name='tissue paper production, virgin')

But when we try to look through the supply chain to understand why we get such high stochastic impacts, we don't get anywhere:

In [9]:
static = bc.LCA({tissue: 1}, ipcc)
static.lci()
static.lcia()

In [10]:
mc = bc.LCA({tissue: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

In [11]:
ba.print_recursive_calculation(
    tissue,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.025,
    _lca_obj=mc,
    _total_score=mc.score,
)

Fraction of score | Absolute score | Amount | Activity
0001 | 208.3 |     1 | 'tissue paper production, virgin' (kilogram, GLO, None)


We seem to be missing something important - the total (stochastic) score is > 200, but the only input above our 2.5% cutoff is sulfite pulp, which is only 3.6% of the total score (specific numbers can change per MC iteration). Maybe this activity has some high direct emissions?

In [12]:
mc.characterized_inventory[:, mc.dicts.activity[tissue.id]].sum()

0.0

We don't see anything in our supply chain, but we can see which activities generate actual impacts by doing a contribution analysis on the final characterized inventory. However, our recursive calculation changed the functional unit while recursing, so we need to "reset" the calculation:

In [13]:
mc = bc.LCA({tissue: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

In [14]:
pd.DataFrame(
    [
        (x, z['name'], z['location']) 
        for x, _, z in ba.ContributionAnalysis().annotated_top_processes(mc)
    ], 
    columns=['Score', 'Name', 'Location']
)

,Score,Name,Location
0,11.093098,palm fruit bunch production,MY
1,6.931211,"heat and power co-generation, lignite",RoW
2,3.691022,"clear-cutting, secondary forest to arable land...",MY
3,3.323755,"heat and power co-generation, natural gas, 200...",RoW
4,3.018054,"heat and power co-generation, hard coal",RU
5,2.714970,"heat and power co-generation, hard coal",PL
6,2.314912,"heat and power co-generation, lignite",PL
7,1.686941,"heat and power co-generation, lignite",RU
8,1.507058,palm fruit bunch production,RoW
9,1.326343,"clear-cutting, primary forest to arable land, ...",MY


[Something is hapening here, but you don't know what it is](https://www.youtube.com/watch?v=we37yX3zpKA)...

Not going to lie, this was very confusing. There are no direct emissions from tissue paper, but also no impacts from the supply chain of tissue paper. Eventually I realized that we can compare the values in the technosphere matrix for the static and Monte Carlo cases and see how different the input amounts are:

In [15]:
def compare_inputs_two_lca_objects(first, second, column_id):
    column_index = first.dicts.activity[column_id]
    results = []
    
    for row in first.technosphere_matrix[:, column_index].tocoo().row:
        a = abs(first.technosphere_matrix[row, column_index])
        b = abs(second.technosphere_matrix[row, column_index])
        try:
            ratio = a / b
        except ZeroDivisionError:
            ratio = 0
        if ratio > 10 or ratio < 0.1:
            act = bd.get_activity(first.dicts.product.reversed[row])
            results.append({
                'ratio': ratio, 
                'a_value': a,
                'b_value': b,
                'name': act['name'],
                'location': act['name'],
                'act_id': act.id,
            })
            
    if results:
        results.sort(key=lambda x: x['ratio'], reverse=True)
        results = pd.DataFrame(results)
    return results

In [16]:
compare_inputs_two_lca_objects(mc, static, tissue.id)

,ratio,a_value,b_value,name,location,act_id
0,1252.096670,16.197410,0.012936,"market for rosin size, for paper production","market for rosin size, for paper production",8907
1,939.879680,9.398797,0.010000,"market for chemical, inorganic","market for chemical, inorganic",14207
2,651.335966,0.651336,0.001000,"market for chemical, organic","market for chemical, organic",5475
3,178.921196,4.842282,0.027064,"market for rosin size, for paper production","market for rosin size, for paper production",6383


We are finally getting somewhere! We have four inputs to the tissue paper where the static amounts are **much less** than the stochastic values sampled. Let's look at one of the uncertainty distributions ([here is the activity dataset at ecoinvent](https://v38.ecoquery.ecoinvent.org/Details/UPR/62960c80-c028-44d4-9d45-a4a6a7a689c6/dd7f13f5-0658-489c-a19c-f2ff8a00bdd9)):

In [17]:
exc = next(exc for exc in tissue.technosphere() if exc.input.id == 8907)
dist = {attr: exc.get(attr) for attr in ('uncertainty type', 'minimum', 'loc', 'maximum')}
dist

{'uncertainty type': 5,
 'minimum': 0.0,
 'loc': 0.0129362298845668,
 'maximum': 80.0}

`5` is the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution), this comes from [stats_arrays](https://stats-arrays.readthedocs.io/en/latest/).

So, what does this mean? Let's look at the statistics:

In [18]:
import stats_arrays as sa

In [19]:
sa.TriangularUncertainty.statistics(sa.TriangularUncertainty.from_dicts(dist))

{'mean': 26.670978743294857,
 'median': 23.436031337921335,
 'mode': 0.0129362298845668,
 'lower': 0.5080001645503973,
 'upper': 71.05645127647223}

When the mean is more than 200 times the static value used, it makes sense that the Monte Carlo results would be significantly biased. And we see a similar pattern in the other three exchanges with high ratios.

Why didn't we see this in our recursive graph traversal? Because that uses the static exchange values instead of the values in the matrix itself:

```python
for exc in activity.technosphere():
```

As of `bw2analyzer` version 0.11.4, we have a modified `print_recursive_calculation` function which will use the actual matrix values (i.e. Monte Carlo samples) instead:

In [20]:
ba.print_recursive_calculation(
    tissue,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.025,
    use_matrix_values=True,
    _lca_obj=mc,
    _total_score=mc.score,
)

Fraction of score | Absolute score | Amount | Activity
0001 | 72.95 |     1 | 'tissue paper production, virgin' (kilogram, GLO, None)
  0.271 | 19.79 | 9.399 | 'market for chemical, inorganic' (kilogram, GLO, None)
    0.271 | 19.79 | 9.399 | 'chemical production, inorganic' (kilogram, GLO, None)
      0.0282 | 2.056 | 0.3133 | 'market for titanium dioxide' (kilogram, RoW, None)
      0.0293 | 2.138 | 0.4283 | 'market for sodium chlorate, powder' (kilogram, RoW, None)
        0.029 | 2.118 | 0.4274 | 'sodium chlorate production, powder' (kilogram, RoW, None)
  0.0313 |  2.28 | 1.015 | 'market for sulfite pulp, bleached' (kilogram, GLO, None)
  0.487 | 35.54 |  16.2 | 'market for rosin size, for paper production' (kilogram, RER, None)
    0.48 | 35.04 |  16.2 | 'rosin size production, for paper production' (kilogram, RER, None)
      0.111 | 8.115 | 4.049 | 'market for sodium hydroxide, without water, in 50% solution state' (k
        0.108 | 7.846 | 4.049 | 'sodium hydroxide to generic

We can now find interesting results more quickly.